In [1]:
from pathlib import Path

from aoc.decorators import timeit

data_file = Path("../Data/day9.txt").read_text()

EXAMPLE = "12345"
EXAMPLE2 = "2333133121414131402"


def prepare(input: str):
    return list(map(int, input.splitlines()[0]))

In [2]:
from dataclasses import dataclass
from typing import Iterable


@dataclass
class DiskItem:
    id: int
    blocks: int
    free_space: int


def make_disk_map(items: list[int]) -> list[DiskItem]:
    buffer: list[int] = []
    chunks: list[DiskItem] = []
    chunk_size = 2
    for i, item in enumerate(items):
        buffer.append(item)
        if (i % chunk_size) == (chunk_size - 1):
            chunks.append(DiskItem(id=len(chunks), blocks=buffer[0], free_space=item))
            buffer = []

    if len(buffer) > 0:
        chunks.append(DiskItem(id=len(chunks), blocks=buffer[0], free_space=0))

    return chunks


def join_string_list(string_list: Iterable[str]):
    return "".join(string_list)


def make_initial_visual_map(disk_map: list[DiskItem]):
    visual_map: list[str] = []
    for disk_item in disk_map:
        visual_map += ([str(disk_item.id)] * disk_item.blocks) + (
            ["."] * disk_item.free_space
        )

    return join_string_list(visual_map)


def compress_disk(visual_map: list[str], *, debug=False):
    if debug:
        print(join_string_list(visual_map))

    last_free_space_index = 0
    for i in reversed(range(len(visual_map))):
        current_item = visual_map[i]
        if current_item == ".":
            continue

        if i < last_free_space_index:
            if debug:
                ...
                # print("🐸 caught up", i)
                # print(join_string_list(visual_map))

            # Yeah I don't care anymore!
            return join_string_list(filter(lambda item: item != ".", visual_map))

        map_item = visual_map[last_free_space_index]
        while map_item != ".":
            last_free_space_index += 1
            if last_free_space_index > (len(visual_map) - 1):
                if debug:
                    # print("🐸 no more free spots")
                    print(join_string_list(visual_map))

                return join_string_list(visual_map)

            map_item = visual_map[last_free_space_index]

        assert map_item == "."

        visual_map[last_free_space_index], visual_map[i] = current_item, map_item
        if debug:
            print(join_string_list(visual_map))

    if debug:
        # print("🐸 ended with glory")
        print(join_string_list(visual_map))

    return join_string_list(visual_map)


@timeit
def part1(input: str, *, debug=False):
    disk_map = make_disk_map(prepare(input))
    visual_map = list(make_initial_visual_map(disk_map))
    compressed_disk = compress_disk(visual_map, debug=debug)

    if debug:
        print(compressed_disk)

    return 0


example_result = part1(EXAMPLE2, debug=True)

assert (
    example_result == 0
), f"Expected example result to be 0, but got {example_result} instead"

result = part1(data_file)

print("result is", result)

assert result == 0

00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
009981118882777333644655556.6.............
0099811188827773336446555566
def part1(input, debug): took: 0.0002 sec
def part1(input, debug): took: 0.0323 sec
result is 0


In [3]:
# @timeit
# def part2(input: str):
#     return 0


# example_result = part2(EXAMPLE)

# assert example_result == 0, f"Expected example result to be 0, but got {example_result} instead"

# result = part2(data_file)

# print("result is", result)

# assert result == 0